### Benign predictions

In [4]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_selection import SelectKBest, mutual_info_classif 
from sklearn.preprocessing import MaxAbsScaler
import numpy as np

X_train = pd.read_csv("feature_updated_dataset_X_2.csv")
y_train = pd.read_csv("../datasets/feature_updated_dataset_y.csv")
X_test = pd.read_csv("benign_dataset_X_2.csv")
y_train = y_train.values.ravel()

scaler = MaxAbsScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

column_names = ['www', 'url_length', 'digit_count', 'percentage_count', 'dot_count',
                'bs_count', 'dash_count', 'url_entropy', 'params_count', 'subdomain_count',
                'domain_extension', 'semicolon_count', 'underscores_count', 'questionmarks_count', 'equals_count', 'ampersands_count', 'digit_letter_ratio', 
                
                'pd_num_count', 'pd_non_alphanumeric_count', 'pd_at_count', 'pd_hyphen_count', 'pd_in_alex_top_1m',
                
                'path_double_slash_count', 'percent20_presence', 'uppercase_dirs', 'single_char_dirs', 'path_count_special_chars',
                'path_zeroes_count', 'path_uppercase_to_lowercase_ratio', 'params_length', 'queries_count']

# model = RandomForestClassifier(n_estimators=50, max_depth=20, random_state=69)
# select_feature_model = SelectKBest(mutual_info_classif, k=3)
# X_train_partition = select_feature_model.fit_transform(X_train, y_train)
# model.fit(X_train_partition, y_train)

# X_test_partition = select_feature_model.transform(X_test)

# y_pred = model.predict(X_test_partition)
# y_test = np.zeros((345738,))
# accuracy = accuracy_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred, average='weighted')

# indexes = select_feature_model.get_support(indices=True)
# print(indexes)
# print(np.array(column_names)[indexes])

# print(f"Accuracy: {accuracy} | F1 score: {f1} | {model.__class__.__name__}")
# print(np.count_nonzero(y_pred == 0)/len(y_pred))
# print(np.count_nonzero(y_pred == 1)/len(y_pred))
# print(np.count_nonzero(y_pred == 2)/len(y_pred))
# print(np.count_nonzero(y_pred == 3)/len(y_pred))

for i in range(1, X_train.shape[1] + 1):
    # Compare current score with previous score and filter out those below previous_score
    best_f1 = 0
    best_features = []

    model = RandomForestClassifier(n_estimators=50, max_depth=20, random_state=69)
    select_feature_model = SelectKBest(mutual_info_classif, k=i)
    X_train_partition = select_feature_model.fit_transform(X_train, y_train)
    model.fit(X_train_partition, y_train)
    X_test_partition = select_feature_model.transform(X_test)
    benign_pred = model.predict(X_test_partition)

    indexes = select_feature_model.get_support(indices=True)
    features_used = np.array(column_names)[indexes]
    
    # Verifying model fit
    y_test = np.zeros((345738,))
    accuracy = accuracy_score(y_test, benign_pred)
    f1 = f1_score(y_test, benign_pred, average='weighted')
    if f1 > best_f1:
        best_f1 = f1
        best_features = features_used

    print(f"Accuracy: {accuracy} | F1 score: {f1} | {model.__class__.__name__} | Number of features: {i} | Features: {features_used}")
    print(np.count_nonzero(benign_pred == 0)/len(benign_pred))
    print(np.count_nonzero(benign_pred == 1)/len(benign_pred))
    print(np.count_nonzero(benign_pred == 2)/len(benign_pred))
    print(np.count_nonzero(benign_pred == 3)/len(benign_pred))

print(f"Best features: {best_features} | F1: {best_f1}")


Accuracy: 0.8093440697869485 | F1 score: 0.8946270455704418 | RandomForestClassifier | Number of features: 1 | Features: ['url_length']
0.8093440697869485
0.15481376070897615
0.012092972134969255
0.023749197369106086


KeyboardInterrupt: 